## 20180402 오늘의 디버깅 일지 

### 1.Quantization 결과 
[-1,1]을 특정 레벨 수로 quantization하는건 좋은데 그 결과가 어느 값으로 맵핑되는지 가끔 헷갈릴 때가 있다 아예 출력을 해놓으면 좋을거 같다

In [42]:
import tensorflow as tf
import numpy as np
for target_level in range(2,17):
    print("******Target level={0}*********".format(target_level),'\n',np.array([i/(target_level-1) for i in range(0,target_level)])*2-1)

******Target level=2********* 
 [-1.  1.]
******Target level=3********* 
 [-1.  0.  1.]
******Target level=4********* 
 [-1.         -0.33333333  0.33333333  1.        ]
******Target level=5********* 
 [-1.  -0.5  0.   0.5  1. ]
******Target level=6********* 
 [-1.  -0.6 -0.2  0.2  0.6  1. ]
******Target level=7********* 
 [-1.         -0.66666667 -0.33333333  0.          0.33333333  0.66666667
  1.        ]
******Target level=8********* 
 [-1.         -0.71428571 -0.42857143 -0.14285714  0.14285714  0.42857143
  0.71428571  1.        ]
******Target level=9********* 
 [-1.   -0.75 -0.5  -0.25  0.    0.25  0.5   0.75  1.  ]
******Target level=10********* 
 [-1.         -0.77777778 -0.55555556 -0.33333333 -0.11111111  0.11111111
  0.33333333  0.55555556  0.77777778  1.        ]
******Target level=11********* 
 [-1.  -0.8 -0.6 -0.4 -0.2  0.   0.2  0.4  0.6  0.8  1. ]
******Target level=12********* 
 [-1.         -0.81818182 -0.63636364 -0.45454545 -0.27272727 -0.09090909
  0.09090909  0.2

### 2. 특정 레벨이 없는 오류 디버깅
![title](./20180402shot01.png)
위처럼 히스토그램을 보면 특정 레벨이 나오지를 않았다, 8레벨의 경우에는 레벨2가 안나왔고
14레벨의 경우에는 레벨4가 안나왔다. 어디서 뭐가 잘린걸까?
사실 이것만 봐도 딱 느낌이왔어야했다. 보나마나 float-int 오가다가 조금씩 잘린거지

그런데 생각보다 삽질을 많이해서 해결하는데 오래 걸렸다. 원인은 아래 코드부분이었다
x는 이미 [-1,1] 구간을 양자화한 결과인데 그거를 다시 역으로 연산하여 level index를 찾으려고 하니까 약간의 손실이 있었던거 같다.
아래처럼 일부 수에 아주 약간
<img src="./20180402shot02.png" width="400" height="400" />


In [61]:
# level_index = (target_level-1)*(x+1)/2
# level_index = tf.cast(level_index,dtype=tf.int32)
a=tf.cast([0.1*i for i in range(1,21)],dtype=tf.int32)
b=tf.to_int32([0.1*i for i in range(1,21)])
c=np.array([0.1*i for i in range(1,21)])
with tf.Session() as sess:
    print([float("%0.1f" % i) for i in c])
    print([float("%0.1f"%i) for i in sess.run(a)])
    print([float("%0.1f"%i) for i in sess.run(b)])
    #아래처럼 0.9999도 결국 0이 되기 때문의 조금의 손실도 치명적이다.
    #앞으로 코딩 할 때는 저런 손실은 어느정도 있다고 보고 저런게 치명적인 부분이면 바로 cast로 넘기지말고 round 같은 함수를 쓰자.

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0]
